In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings

# Ignorar warnings de convergencia
warnings.filterwarnings("ignore")

# 1. Cargar y agregar por producto y mes
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df = df.groupby(["product_id", "periodo"])["tn"].sum().reset_index()
df = df.sort_values(["product_id", "periodo"])

# 2. Leer universo de productos
productos_objetivo = pd.read_csv("product_id_apredecir201912.txt")["product_id"].tolist()

# 3. Inicializar
resultados = []
entrenados = 0
fallbacks = 0

# 4. Loop por producto
for producto in productos_objetivo:
    data_prod = df[(df["product_id"] == producto) & (df["periodo"] < "2020-01-01")].copy()
    data_prod = data_prod.sort_values("periodo")
    ts = data_prod.set_index("periodo")["tn"]

    try:
        if ts.shape[0] >= 6:
            modelo = ARIMA(ts, order=(1, 1, 0))
            modelo_fit = modelo.fit()
            pred = modelo_fit.forecast(steps=2)[-1]
            resultados.append({"product_id": producto, "tn_predicho": pred})
            entrenados += 1
            continue
    except Exception as e:
        print(f"⚠️ ARIMA falló para producto {producto}: {e}")

    # Fallback combinado
    ultimos_12 = ts.tail(12)
    prom_ultimos12 = ultimos_12.mean() if not ultimos_12.empty else 0.0
    febreros = ts[ts.index.month == 2]
    prom_febreros = febreros.mean() if not febreros.empty else 0.0
    pred = 0.6 * prom_ultimos12 + 0.4 * prom_febreros

    resultados.append({"product_id": producto, "tn_predicho": pred})
    fallbacks += 1

# 5. Exportar
df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("predicciones_febrero2020_arima_780_combinado_AGREGADO.csv", index=False)

print("✅ Archivo generado: predicciones_febrero2020_arima_780_combinado_AGREGADO.csv")
print(f"🟢 Modelos ARIMA entrenados: {entrenados}")
print(f"🟡 Fallbacks aplicados: {fallbacks}")
print(f"🔢 Total productos: {len(df_resultado)}")


⚠️ ARIMA falló para producto 20089: -1
⚠️ ARIMA falló para producto 20192: -1
⚠️ ARIMA falló para producto 20313: -1
⚠️ ARIMA falló para producto 20426: -1
⚠️ ARIMA falló para producto 20456: -1
⚠️ ARIMA falló para producto 20469: -1
⚠️ ARIMA falló para producto 20596: -1
⚠️ ARIMA falló para producto 20641: -1
⚠️ ARIMA falló para producto 20666: -1
⚠️ ARIMA falló para producto 20786: -1
⚠️ ARIMA falló para producto 20793: -1
⚠️ ARIMA falló para producto 20824: -1
⚠️ ARIMA falló para producto 20879: -1
⚠️ ARIMA falló para producto 20885: -1
⚠️ ARIMA falló para producto 20936: -1
⚠️ ARIMA falló para producto 21049: -1
⚠️ ARIMA falló para producto 21170: -1
⚠️ ARIMA falló para producto 21190: -1
⚠️ ARIMA falló para producto 21200: -1
✅ Archivo generado: predicciones_febrero2020_arima_780_combinado_AGREGADO.csv
🟢 Modelos ARIMA entrenados: 737
🟡 Fallbacks aplicados: 43
🔢 Total productos: 780
